In [95]:
import sys
sys.path.append('..')
# reload local packages automatically
%load_ext autoreload
%autoreload 2

from jax import config
config.update("jax_enable_x64", True)

# Import packages.
import cvxpy as cp
import numpy as np

import matplotlib.pyplot as plt


from opentn.transformations import create_kitaev_liouvillians, exp_operator_dt, factorize_psd, super2choi
d, N = 2, 4
dim = d**N
Lvec, Lvec_odd, Lvec_even, Lnn = create_kitaev_liouvillians(N=N, d=d, gamma=1e-2)
tau = 4
superops_exp = []
for i, op in enumerate([Lvec, Lvec_odd, Lvec_even]):
    if i == 1:
        superops_exp.append(exp_operator_dt(op, tau/2, 'jax'))
    else:
        superops_exp.append(exp_operator_dt(op, tau, 'jax'))
exp_Lvec, exp_Lvec_odd, exp_Lvec_even = superops_exp

X1 = factorize_psd(psd=super2choi(exp_Lvec_odd), tol=1e-12)
X2 = factorize_psd(psd=super2choi(exp_Lvec_even), tol=1e-12)

from opentn.optimization import gds, frobenius_norm, model_Ys, compute_loss
cost_n4, grads_n4, xi_n4 = gds(fn=compute_loss, x0=[X1, X2, X1], iter=int(2e3), loss_fn=frobenius_norm, model=model_Ys, rate=1.5e-9, exact = exp_Lvec, show_cost=False)
print(cost_n4[0])
print(cost_n4[-1])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2.6383007149418305e-07
1.4977787039695335e-07


In [96]:
from opentn.transformations import super2choi, choi_composition
from opentn.optimization import small2zero
from scipy import sparse

with open('Y1_scs_warm_gds_eps_e-12.npy', 'rb') as f:
    Y1_cvxpy = np.load(f)
# rest of matrices optimized with GSD
X1_gds, X2_gds, X3_gds = xi_n4[-1]
Y1_gds = X1_gds@X1_gds.conj().T
Y2_gds = X2_gds@X2_gds.conj().T
Y3_gds = X3_gds@X3_gds.conj().T

tol = 1e-12

# rhs to which we want to approximate
rhs = np.asarray(super2choi(superop=exp_Lvec, dim=dim))
# srhs = sparse.csr_matrix(rhs) # small2zero was deleted
srhs = sparse.csr_matrix(small2zero(rhs, tol=tol)) # small2zero was deleted
srhs = srhs.astype(np.float64)
# change partial transpose to A
C2 = choi_composition(C1=small2zero(Y2_gds, tol=tol), C2=small2zero(Y3_gds, tol=tol), dim=dim) # small2zero was deleted
print(frobenius_norm(choi_composition(small2zero(Y1_cvxpy, tol=tol), C2), small2zero(rhs, tol=tol))) # value when small2zero was used on A and rhs: 2.6522855638959974e-07
# update: apply small2zero to both A and rhs with same tol=1e-10

9.518505947521337e-08


/opt/miniconda3/envs/thesis/lib/python3.10/site-packages/scipy/sparse/_data.py:72: ComplexWarning: Casting complex values to real discards the imaginary part
  self.data.astype(dtype, casting=casting, copy=True),


In [98]:
# now lets try an optimization for the third layer using as a warm start the solution that we found with GDS as well.
# now C1 = Y2 @ Y1 and C2 = Y3_opt
from opentn.transformations import link_product_cvxpy
Y3_cvxpy = cp.Variable((dim**2,dim**2), PSD=True)
lhs = link_product_cvxpy(C1=choi_composition(small2zero(Y1_cvxpy, tol), small2zero(Y2_gds, tol)), C2=Y3_cvxpy, dim=dim, transpose=0, optimization=True)
cost = cp.norm(lhs - srhs, "fro")
prob = cp.Problem(cp.Minimize(cost))
Y3_cvxpy.value = small2zero(Y3_gds, tol=tol).astype(np.float64) 
cost.value

/opt/miniconda3/envs/thesis/lib/python3.10/site-packages/scipy/sparse/_data.py:72: ComplexWarning: Casting complex values to real discards the imaginary part
  self.data.astype(dtype, casting=casting, copy=True),
/var/folders/lv/986s3mp97gv9n3rb757bg87m0000gn/T/ipykernel_4057/2425024391.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  Y3_cvxpy.value = small2zero(Y3_gds, tol=tol).astype(np.float64)


9.518505951367502e-08

In [99]:
prob.solve(solver=cp.SCS, verbose=True, canon_backend=cp.SCIPY_CANON_BACKEND, eps=tol)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jul 29 10:02:21 PM: Your problem has 65536 variables, 0 constraints, and 0 parameters.
(CVXPY) Jul 29 10:02:21 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 29 10:02:21 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 29 10:02:21 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 29 10:02:21 PM: Compiling problem (target solver=SCS).
(CVXPY) Jul 29 10:02:21 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

/opt/miniconda3/envs/thesis/lib/python3.10/site-packages/cvxpy/problems/problem.py:1387: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Jul 29 10:28:24 PM: Problem status: optimal_inaccurate
(CVXPY) Jul 29 10:28:24 PM: Optimal value: 9.518e-08
(CVXPY) Jul 29 10:28:24 PM: Compilation took 6.223e+02 seconds
(CVXPY) Jul 29 10:28:24 PM: Solver (including time spent in interface) took 9.386e+02 seconds


9.518374307487138e-08

In [102]:


max_idx = 16
for op in [super2choi(exp_Lvec_odd), X1@X1.conj().T, Y3_gds, Y3_cvxpy.value, small2zero(Y3_cvxpy.value, tol=tol)]:
    print(np.linalg.matrix_rank(op, tol=tol))
    print(np.sort(np.linalg.svd(op, compute_uv=False))[::-1][:max_idx])
    print('----')

4
[1.59601247e+01 1.99252079e-02 1.99252079e-02 2.48753638e-05
 9.14980688e-15 8.31005402e-15 8.28778104e-15 7.68898909e-15
 5.05313981e-15 4.30617828e-15 4.16892000e-15 3.72903726e-15
 3.42133722e-15 3.09273854e-15 2.96987622e-15 2.80821003e-15]
----
4
[1.59601247e+01 1.99252079e-02 1.99252079e-02 2.48753638e-05
 7.95281221e-15 4.45619364e-15 4.15294601e-15 3.74630065e-15
 3.38415367e-15 3.00980011e-15 2.78457911e-15 2.75926410e-15
 2.62875443e-15 2.57327522e-15 2.56723560e-15 2.12727961e-15]
----
4
[1.59601248e+01 1.99251811e-02 1.99251788e-02 2.48753818e-05
 6.16326090e-15 5.79055785e-15 4.75687568e-15 4.40982071e-15
 4.01009468e-15 3.97914112e-15 3.83097705e-15 3.47001439e-15
 3.42981826e-15 3.00149193e-15 2.99661371e-15 2.79375070e-15]
----
12
[1.59601248e+01 1.99251810e-02 1.99251788e-02 2.48753728e-05
 2.55544033e-10 7.48022052e-11 6.07035791e-11 5.99426383e-11
 2.53479161e-11 5.87060798e-12 2.36016500e-12 1.46792739e-12
 3.95953541e-13 8.44573067e-14 8.03467510e-14 7.78232283e-

In [108]:
# are the first and third layer the same after this new optimization?
np.allclose(Y1_cvxpy, Y3_cvxpy.value, atol=tol), 2*np.linalg.norm(Y1_cvxpy - Y3_cvxpy.value)/(np.linalg.norm(Y3_cvxpy.value)+np.linalg.norm(Y1_cvxpy))

(False, 1.6256790070368328e-08)

In [109]:
with open(f'Y3_scs_warm_gds_eps_{tol}.npy', 'wb') as f:
    np.save(f, Y3_cvxpy.value)

In [40]:
# this si the one where the small2zero was not apply everywhere
prob.solve(solver=cp.SCS, verbose=True, canon_backend=cp.SCIPY_CANON_BACKEND, eps=0.25e-10)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jul 26 02:12:35 PM: Your problem has 65536 variables, 0 constraints, and 0 parameters.
(CVXPY) Jul 26 02:12:35 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 26 02:12:35 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 26 02:12:35 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 26 02:12:35 PM: Using cached ASA map, for faster compilation (bypassing reduction chain).
(CVXPY) Jul 26 02:12:35 PM: Finished problem compilation (t

9.518053924910776e-08

In [ ]:
# eps: 5e-10: 9.517969895842572e-08
# eps: 1e-10: 9.518041680165088e-08
# eps: 0.5e-10: 9.517904309118482e-08
